# TER percentiles (bar-plot)
This notebook uses the Landscape Model Python API to access and read from a data store containing simulation results. It shows step-by-step how to prepare and conduct data retrieval and analysis of TER values for generating a simple bar-plot using _matplotlib_.

First we have to import some Landscape Module Python modules that were made available to the notebook's Python instance.

The _observer_ namespace contains some pre-defined observers that monitor everything going on in the Landscape Model and are able to react to it in some way. We import the module because we will need an observer that gives us feedback during data access.

In [ ]:
import observer

The _stores_ namespace contains a set of implemented stores that manage the access to persisted data. We need this module because we want some encapsulated access to the data of interest.

In [ ]:
import stores

The _base_ namespace contains class definitions of Landscape Model base objects. We need to import this module if we like to make use of the functionality implemented in these classes.

In [ ]:
import base

The _attrib_ namespace holds a number of classes that specify, check and possibly transpose different aspects of data, i.e. the metadata of the data. Ths module is required if we want to make use of the automatic checks and capabilities regarding the semantics of data.

In [ ]:
import attrib

We will retrieve the data as numpy arrays from the data store and will continue to process them as numpy arrays. To use the according functions, we have to import the _numpy_ module.

In [ ]:
import numpy as np

Plotting will be done here using _matplotlib.pyplot_ which is the last import of this notebook.

In [ ]:
import matplotlib.pyplot as plt

To monitor and react on the state of Landscape Model objects, we can use an observer. Observers are implementations that monitor the processing, data access, data transformations, etc. of Landscape Model classes and react to specific events, e.g. by outputting messages to the console or logfile, by starting analysis of simulated data, by cleaning up data directories, etc.

We instantiate a `ConsoleObserver` here that writes messages to the standard output (usually the console terminal) in order to keep track of the data retrieval in detail. For usage in a Jupyter Notebook, the `print_output` argument has to be set to `True`.

In [ ]:
console_observer = observer.ConsoleObserver(print_output=True)

Next we instantiate a store. A store is a Landscape Model object that encapsulates the communication with a low-level database that contains persisted data in a Landscape Model compatible representation.

We use a `X3dfStore` here that works against a local x3df file. x3df is a HDF5-based format developed to contain data and metadata from a Landscape Model. A sample x3df file database is included in the _sample_ sub-folder of the notebook-containing folder and is given here as relative path. Other x3df files can be loaded by specifying their absolute path. We pass the previously created ConsoleObserver to the X3dfStore so that messages produced during store interaction will be written to the notebook. Finally, we decide for access mode `"r"` (read-only) as we are not going to modify the data in the store here. This has the benefit of avoiding unintentional altering of the data and being able to read from the same database file from other processes simultaneously.

In [ ]:
x3df_store = stores.X3dfStore("sample/store", console_observer, "r")

Data is exchanged in the Landscape Model always by connecting an `Input` to an `Output`. In the store, the data is only persisted once as given by an output. This does, however, not mean that the data of an input and that of its connected output are identical: the separation of input and output allows the data to have different specification, represented by data attributes and stored as metadata. During the retrieval of data, the Landscape Model checks the congruency of data specifications between output (=the stored data) and the input (=the data requested, e.g. by the user) and reacts on discrepancies by warning messages, automatic data transformations to fit the input specification, etc.

This means, that a data client has to specify data as an input before the actual data can be retrieved and represented a Python object. The level of detail by which the input is specified, is however very flexible, and it is up to the user to decide in how much detail this specification takes place ("opt-in specification").

In the following, we create an `Input` with the name `"DistanceGroups"`. The input is specified by the following additional attributes: it should be represented as a Python class of an `numpy.ndarray` and it should be given by the scales `"space_x/1sqm, space_y/1sqm"`. We pass in the previously instantiated ConsoleObserver so that any messages generated by accessing the input are written to the Jupyter notebook. Finally, we state that input is connected to an `Output` with the name `"/LandscapeScenario/analysis_distance_groups_values"` that is stored in the previously created X3dfStore.

Values read from the input will so be Numpy arrays representing with values representing 1 square-meter cells, given that this specification is compatible (i.e. identical or transformable from) the data as specified by the output.

The distance group values specified here will indicate up to how much meter away from the nearest field edge an individual square-meter cell is located.

In [ ]:
distance_groups_input = base.Input(
    "DistanceGroups", 
    (attrib.Class(np.ndarray), attrib.Scales("space_y/1sqm, space_x/1sqm")),
    console_observer, 
    base.Output("LandscapeScenario/analysis_distance_groups_values", x3df_store)
)

Another output we need for analysis are the actual TER values, stored in the sample database under `"/TerRQ/TER"`. In addition to the DistanceGroups input, we specify here that the values should have no physical unit (`'1'` as default symbol for dimensionless values) and are given at the scales `"time/day, space_x/1sqm, space_y/1sqm"`, i.e. we can expect a three-dimensional Numpy aray here where the first axis moves along individual days.

In [ ]:
ter_input = base.Input(
    "TER", 
    (attrib.Class(np.ndarray), attrib.Unit("1"), attrib.Scales("space_y/1sqm, space_x/1sqm, time/day")), 
    console_observer, 
    base.Output('TerRQ/TER', x3df_store)
)

With the inputs properly specified, we can finally retrieve data from the input using their `read` functions. Large datasets should preferably be processed in chunks to keep the memory footprint reasonably low, but for the small sample dataset this is not an issue. However, we can still use the mechanism to read only a part of the data (using the keyword `slices`) to read only the values of the habitat part of the sample data, which is everything having (zero-based) x-coordinates of 100 and larger.

The following code will load the distance groups for the 100 m x 100 m habitat portion of the sample data into a 2d Numpy array. The messages produced during data retrieval (and processed by the ConsoleObserver) should indicate that the data requested fits the specification of the stored data.

In [ ]:
distance_groups = distance_groups_input.read(slices=(slice(0, 100), slice(100, 200))).values

The retrieval of the TER data works exactly the same, except that we have also to specify the slice for the time-axis when using the `slices` keyword. We will load here only data for the first day (the day of application and the only day stored in the sample data set). As there is now only one element at the time-axis selected Numpy will automatically cast the result into a 2d array, meaning that the resulting array will have exactly the same shape as the one of the distance groups.

In [ ]:
ter = ter_input.read(slices=(slice(0, 100), slice(100, 200), 0)).values

Now all the data is read in, and we can start with the actual analysis. First, though, we have to do some data preparation: For square-meters where there is no exposure, the TER is reported as 0, but we want to have a high value instead here, so we replace all TER values of 0 by 100.

In [ ]:
ter[ter == 0] = 100

Next, we define a lis of distances we like to consider. These should be interpreted as 'habitat up to x meter away from the field edge'. `'inf'` thereby means any distance from a field edge, which should be identical to 100 m in this example.

In [ ]:
distances = [10, 20, 50, 100, float("inf")]

Bringing the TER values, the distance groups and the pre-defined distances together, we calculate now the 10th percentile of TER values for every distance group.

In [ ]:
quantiles = [np.quantile(ter[distance_groups <= d], 0.1) for d in distances]

The calculated percentiles look as the following:

In [ ]:
quantiles

For preparation of plotting, we define a list of RGB colors in which the individual distance groups should be represented.

In [ ]:
colors = [
    ( 27/255,  57/255,  50/255), 
    ( 45/255, 106/255,  79/255), 
    ( 64/255, 145/255, 108/255), 
    ( 82/255, 183/255, 136/255), 
    (116/255, 198/255, 157/255)
]

Finally, a simple bar-plot visualization using _matplotlib_.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.bar(range((len(distances))), quantiles, color=colors)
ax.set(title="10$^{th}$ percentile of TER(ER$_{50}$) - no mitigation", ylabel="TER [-]", xlabel="distance group")
ax.xaxis.set(ticks=range(len(distances)), ticklabels=distances)
plt.show()